In [157]:
import numpy as np
import math
import residuals as r
np.set_printoptions(threshold=40)

In [93]:
# I am defining the weighted standard deviation according to this definition:
# https://en.wikipedia.org/wiki/Weighted_arithmetic_mean#Weighted_sample_variance
# I am using the inverse square of the error arrays to define the weights

factor10 = math.sqrt((25*60)/10) 
factor80 = math.sqrt((25*60)/80) 

def weighted_stddev(values, errors):
    '''
    Return the weighted average and standard deviation.

    '''
    weights = 1 / ((errors)**2)
    average = np.average(values, weights=weights)
    variance = np.average((values-average)**2, weights=weights)  # Fast and numerically precise
    return (math.sqrt(variance))

def jitter_ratio(dumpresids_file, condition_lower, condition_upper, titlearg):
    '''
    Return the ratio of sigma_dump over sigma_all.

    '''
    
    #Define sigma_all (or std_all) as entire observation period
    x=r.read_residuals(filename=dumpresids_file)

    condition = (x.bary_TOA > condition_lower) & (x.bary_TOA < condition_upper)
    Resid_all = x.prefit_sec[condition]
    Errors_all = x.uncertainty[condition]
    std_all = weighted_stddev(Resid_all, Errors_all)
    
    ##============
    
    #Define sigma_dump (or std_all) as individual dumps within observation period
    
    #This isolates the numbers for the different dump times
    rounded = np.round(np.array([x.bary_TOA - 0.00005]), 4)
    dumptimes=[]
    for i in rounded.tolist()[0]: 
        if i not in dumptimes:
            dumptimes.append(i)

    #This calculates a list of sigma_dump values and avearages those
    dump_devs=[]
    for i in dumptimes:
        timeindex = (rounded == i)[0]
        Resid = x.prefit_sec[timeindex]
        Errors = x.uncertainty[timeindex]
        single_std_dump = weighted_stddev(Resid, Errors)
        all_dumps.append(single_std_dump)
    std_dump=np.average(dump_devs)
    
    #Setup jitter ratio
    ratio = std_dump / std_all
    
    print titlearg
    print std_dump
    print "jitter ratio = " + str(ratio)   
    
    

def wrong_jitter_ratio(allresids_file, dumpresids_file, condition_lower, condition_upper, titlearg, int_time):
    '''
    Return the ratio of sigma_dump over sigma_all.

    '''
    
    #Define sigma_all (or std_all)
    y=r.read_residuals(filename=allresids_file)
    FullResid = y.prefit_sec
    FullErrors = y.uncertainty
    std_all = weighted_stddev(FullResid, FullErrors)
    
    if int_time is 10:
        std_all = (factor10)*std_all
    
    if int_time is 80:
        std_all = (factor80)*std_all
    
    #=======
    
    #Define sigma_dump (or std_dump)
    x=r.read_residuals(filename=dumpresids_file)

    condition = (x.bary_TOA > condition_lower) & (x.bary_TOA < condition_upper)
    Resid = x.prefit_sec[condition]
    Errors = x.uncertainty[condition]
    std_dump = weighted_stddev(Resid, Errors)
    
    #Setup jitter ratio
    ratio = std_dump / std_all
    
    print titlearg
    print std_dump
    print "jitter ratio = " + str(ratio)   

In [175]:
# working on getting sigma dump to be individual TOA dumps, using J1713 MJD 56380 first
#Doing this by isolating to the fourth sigfig and rounding down

x=r.read_residuals(filename="/nimrod1/eschwab/residuals/J1713+0747_resid_56380_NTF8.tmp")

#This rounds it down to the fourth digit after the decimal and makes it a python list
rounded = np.round(np.array([x.bary_TOA - 0.00005]), 4)

#This creates an index for residuals and uncertainty where rounded equals a specific number
timeindex = (rounded == 56380.3514)[0]
x.prefit_sec[timeindex]
x.uncertainty[timeindex]

##========== Above not needed

#This isolates the numbers for the different dump times
rounded = np.round(np.array([x.bary_TOA - 0.00005]), 4)
dumptimes=[]
for i in rounded.tolist()[0]: 
    if i not in dumptimes:
        dumptimes.append(i)

#This calculates a list of sigma dump values and avearages those
all_dumps=[]
for i in dumptimes:
    timeindex = (rounded == i)[0]
    Resid = x.prefit_sec[timeindex]
    Errors = x.uncertainty[timeindex]
    single_std_dump = weighted_stddev(Resid, Errors)
    all_dumps.append(single_std_dump)
std_dump=np.average(all_dumps)

In [92]:
#Checking order of operations

print 25*60
print 1500/10
print math.sqrt(150)
print math.sqrt((25*60)/10)

print math.sqrt((25*60)/80)

#yes, order of operations works

1500
150
12.2474487139
12.2474487139
4.24264068712


In [80]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56380.tmp",
             "/nimrod1/eschwab/residuals/J1713+0747_resid_56380_80F8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 80 second subintervals', 80)

J1713+0747, MJD 56380, 80 second subintervals
2.44415585816e-07
jitter ratio = 0.0602632499995


In [81]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56380.tmp",
             "/nimrod1/eschwab/residuals/J1713+0747_resid_56380_NTF8.tmp",
            56380.35, 56380.367, 'J1713+0747, MJD 56380, 10 second subintervals', 10)

J1713+0747, MJD 56380, 10 second subintervals
5.82671845721e-07
jitter ratio = 0.0497666388099


In [82]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56201.tmp",
             "/nimrod1/eschwab/residuals/J1713+0747_resid_56201_80F8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 80 second subintervals', 80)

J1713+0747, MJD 56201, 80 second subintervals
1.73501235065e-07
jitter ratio = 0.0793011729408


In [83]:
jitter_ratio("/nimrod1/eschwab/residuals/residJ1713_-56201.tmp",
             "/nimrod1/eschwab/residuals/J1713+0747_resid_56201_NTF8.tmp",
            56201.85, 56201.872, 'J1713+0747, MJD 56201, 10 second subintervals', 10)

J1713+0747, MJD 56201, 10 second subintervals
5.43224438005e-07
jitter ratio = 0.0860096068797


In [84]:
jitter_ratio("/nimrod1/eschwab/residuals/residB1937_-56465.tmp",
             "/nimrod1/eschwab/residuals/B1937+21_resid_56465_80F8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 80 second subintervals', 80)

B1937+21, MJD 56465, 80 second subintervals
4.68059107539e-08
jitter ratio = 0.0277180473429


In [85]:
jitter_ratio("/nimrod1/eschwab/residuals/residB1937_-56465.tmp",
             "/nimrod1/eschwab/residuals/B1937+21_resid_56465_NTF8.tmp",
            56465.3, 56465.32, 'B1937+21, MJD 56465, 10 second subintervals', 10)

B1937+21, MJD 56465, 10 second subintervals
7.05138954322e-08
jitter ratio = 0.0144652939138
